<a href="https://colab.research.google.com/github/iGesora/Datasets/blob/master/Telematics_Cleaning_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive  #mount your Google Drive to Colab
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
%%time
df = pd.read_csv('/content/drive/MyDrive/ORANGEEV FILES/2023_May_Telematics_SOC_Merged.csv')

CPU times: user 1.53 s, sys: 176 ms, total: 1.71 s
Wall time: 1.99 s


In [ ]:
df.shape

(1670269, 4)

In [ ]:
df.head()

,S/N,Machine Type,Event Time (Local),Value Name,Tag,Value,Delta Value,Raw Value
0,122792,NaN,"2023/05/01, 0:00am",State of Charge (SOC),SOC,100.0,NaN,10000
1,121613,NaN,"2023/05/01, 0:00am",State of Charge (SOC),SOC,100.0,NaN,10000
2,121607,NaN,"2023/05/01, 0:00am",State of Charge (SOC),SOC,100.0,NaN,10000
3,122874,NaN,"2023/05/01, 0:00am",State of Charge (SOC),SOC,100.0,NaN,10000
4,122872,NaN,"2023/05/01, 0:00am",State of Charge (SOC),SOC,100.0,NaN,10000


In [ ]:
df.tail()

,S/N,Machine Type,Event Time (Local),Value Name,Tag,Value,Delta Value,Raw Value
1711501,121565,NaN,"2023/05/31, 11:59pm",State of Charge (SOC),SOC,100.00,NaN,10000
1711502,121911,NaN,"2023/05/31, 11:59pm",State of Charge (SOC),SOC,77.63,NaN,7763
1711503,122796,NaN,"2023/05/31, 11:59pm",State of Charge (SOC),SOC,77.42,NaN,7742
1711504,121914,NaN,"2023/05/31, 11:59pm",State of Charge (SOC),SOC,100.00,NaN,10000
1711505,122811,NaN,"2023/05/31, 11:59pm",State of Charge (SOC),SOC,84.30,NaN,8430


In [ ]:
%%time
# Check for null values in all columns
df.isnull().sum()


CPU times: user 795 ms, sys: 7.78 ms, total: 802 ms
Wall time: 822 ms


S/N                         0
Machine Type          1711506
Event Time (Local)          0
Value Name                  0
Tag                         0
Value                       0
Delta Value           1711506
Raw Value                   0
dtype: int64

In [ ]:
# List of columns to drop
columns_to_drop = ['Machine Type', 'Value Name', 'Tag', 'Delta Value', 'Raw Value']

# Drop the specified columns
df = df.drop(columns=columns_to_drop)

In [ ]:
df.head(10)

,S/N,Event Time (Local),Value
0,122792,"2023/05/01, 0:00am",100.00
1,121613,"2023/05/01, 0:00am",100.00
2,121607,"2023/05/01, 0:00am",100.00
3,122874,"2023/05/01, 0:00am",100.00
4,122872,"2023/05/01, 0:00am",100.00
5,121358,"2023/05/01, 0:00am",94.16
6,121757,"2023/05/01, 0:00am",100.00
7,120740,"2023/05/01, 0:00am",53.51
8,123337,"2023/05/01, 0:00am",96.06
9,121689,"2023/05/01, 0:00am",95.77


In [ ]:
# Define the mapping of old column names to new column names
column_mapping = {
    'S/N': 'Truck Serial Number',
    'Event Time (Local)': 'Timestamp',
    'Value': 'SoC'
}

# Rename the columns
df = df.rename(columns=column_mapping)

# Drop the index column
df = df.reset_index(drop=True)

#view new columns
df.columns

Index(['Truck Serial Number', 'Timestamp', 'SoC'], dtype='object')

In [ ]:
%%time
# Convert the timestamp column to datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Extract the desired components into new columns
df['Date'] = df['Timestamp'].dt.strftime('%Y-%m-%d')  # Short format date, e.g., '2023-06-04'
df['Time (24hr)'] = df['Timestamp'].dt.strftime('%H:%M:%S')  # Time in 24-hour format, e.g., '15:30:45'

# Drop the original timestamp column
df = df.drop(columns=['Timestamp'])

df.head(5)

CPU times: user 13.7 s, sys: 192 ms, total: 13.9 s
Wall time: 14 s


,Truck Serial Number,SoC,Date,Time (24hr)
0,122792,100.0,2023-05-01,00:00:00
1,121613,100.0,2023-05-01,00:00:00
2,121607,100.0,2023-05-01,00:00:00
3,122874,100.0,2023-05-01,00:00:00
4,122872,100.0,2023-05-01,00:00:00


In [ ]:
unique_serial_numbers = len(df['Truck Serial Number'].unique())
print("Unique number of truck serial numbers:", unique_serial_numbers)

Unique number of truck serial numbers: 810


In [ ]:
%%time
# Sort the dataframe by 'Truck Serial Number', 'Date', and 'Time (24hr)'
df = df.sort_values(by=['Truck Serial Number', 'Date', 'Time (24hr)'])

# Identify duplicate rows based on the specified columns for each unique truck serial number
duplicate_rows = df[df.duplicated(subset=['Truck Serial Number', 'SoC', 'Date', 'Time (24hr)'], keep=False)]

if duplicate_rows.empty:
    print("No duplicate values found for the specified columns for any truck serial number.")
else:
    # Drop duplicate rows and keep only the first occurrence
    df = df.drop_duplicates(subset=['Truck Serial Number', 'SoC', 'Date', 'Time (24hr)'], keep='first')

    print("\nDuplicate rows:")
    print(duplicate_rows)
    print("\nCount of duplicate rows:", len(duplicate_rows))

# Print the modified dataframe without duplicate rows
print("\nModified dataframe:")
print(df)



Duplicate rows:
         Truck Serial Number    SoC        Date Time (24hr)
7580                  120660  80.12  2023-05-01    03:38:00
7593                  120660  80.12  2023-05-01    03:38:00
7609                  120660  80.12  2023-05-01    03:38:00
37283                 120660  96.21  2023-05-01    16:59:00
37290                 120660  96.21  2023-05-01    16:59:00
...                      ...    ...         ...         ...
1699802               151829  44.44  2023-05-31    19:16:00
1699804               151829  44.44  2023-05-31    19:16:00
1703949               151829  51.83  2023-05-31    21:03:00
1703951               151829  51.83  2023-05-31    21:03:00
1703975               151829  51.83  2023-05-31    21:03:00

[80122 rows x 4 columns]

Count of duplicate rows: 80122

Modified dataframe:
         Truck Serial Number    SoC        Date Time (24hr)
14                    120660  92.85  2023-05-01    00:00:00
1098                  120660  92.53  2023-05-01    00:15:00
1560

In [ ]:
df.shape

(1670269, 4)

In [ ]:
df.columns

Index(['Truck Serial Number', 'SoC', 'Date', 'Time (24hr)'], dtype='object')

In [ ]:
df.head(10)


,Truck Serial Number,SoC,Date,Time (24hr)
14,120660,92.85,2023-05-01,00:00:00
1098,120660,92.53,2023-05-01,00:15:00
1560,120660,92.31,2023-05-01,00:30:00
2048,120660,92.10,2023-05-01,00:45:00
2504,120660,91.90,2023-05-01,01:00:00
2978,120660,91.68,2023-05-01,01:15:00
3471,120660,90.67,2023-05-01,01:30:00
3951,120660,89.22,2023-05-01,01:45:00
4264,120660,88.10,2023-05-01,01:55:00
4447,120660,87.60,2023-05-01,02:00:00


In [ ]:
df.tail(10)

,Truck Serial Number,SoC,Date,Time (24hr)
1705761,151829,58.38,2023-05-31,21:48:00
1706338,151829,60.51,2023-05-31,22:03:00
1706930,151829,62.65,2023-05-31,22:18:00
1707490,151829,64.83,2023-05-31,22:33:00
1708085,151829,66.98,2023-05-31,22:48:00
1708677,151829,69.11,2023-05-31,23:03:00
1709264,151829,71.25,2023-05-31,23:18:00
1709865,151829,73.45,2023-05-31,23:33:00
1710469,151829,75.65,2023-05-31,23:48:00
1711226,151829,76.57,2023-05-31,23:55:00


In [ ]:
%%time
###########PERFECT!!##########

# Sort the dataframe by 'Truck Serial Number', 'Date', and 'Time (24hr)'
df = df.sort_values(by=['Truck Serial Number', 'Date', 'Time (24hr)'])

# Initialize lists for new columns
truck_serial_numbers = []
soc_list = []
soc_kwh_list = []
date_list = []
time_list = []
charging_list = []
discharging_list = []
idle_list = []
change_in_soc_list = []
change_in_soc_kwh_list = []
prev_truck_serial_number = None
prev_date = None
prev_soc = None
prev_time = None

# Iterate over the rows in the dataframe
for index, row in df.iterrows():
    truck_serial_number = row['Truck Serial Number']
    soc = row['SoC']
    time = pd.to_datetime(row['Time (24hr)'])
    date = row['Date']

    # Check for truck serial number change
    if truck_serial_number != prev_truck_serial_number:
        prev_truck_serial_number = truck_serial_number
        prev_soc = None
        prev_time = None

    # Check for date change
    if date != prev_date:
        prev_date = date
        prev_soc = None
        prev_time = None

    # Calculate time difference in minutes
    if prev_time is not None:
        time_diff = (time - prev_time).total_seconds() / 60
    else:
        time_diff = 0

    # Calculate change in SoC
    if prev_soc is not None:
        change_in_soc = soc - prev_soc
    else:
        change_in_soc = None

    # Assign time difference to corresponding columns based on change in SoC
    if change_in_soc is not None:
        if change_in_soc < 0:
            discharging_list.append(time_diff)
            charging_list.append(0)
            idle_list.append(0)
        elif change_in_soc > 0:
            discharging_list.append(0)
            charging_list.append(time_diff)
            idle_list.append(0)
        else:
            discharging_list.append(0)
            charging_list.append(0)
            idle_list.append(time_diff)
    else:
        discharging_list.append(0)
        charging_list.append(0)
        idle_list.append(0)

    # Append truck serial number, SoC, date, time, and change in SoC to the lists
    truck_serial_numbers.append(truck_serial_number)
    soc_list.append(soc)
    soc_kwh_list.append(soc * 1.8)
    date_list.append(date)
    time_list.append(time)
    change_in_soc_list.append(change_in_soc)

    # Update previous values
    prev_soc = soc
    prev_time = time

# Calculate Change in SoC kWh
change_in_soc_kwh_list = [change * 1.8 if change is not None else 0 for change in change_in_soc_list]

# Create the updated dataframe with the desired column order
updated_df = pd.DataFrame({
    'Truck Serial Number': truck_serial_numbers,
    'SoC': soc_list,
    'SoC in kWh': soc_kwh_list,
    'Date': date_list,
    'Time (24hr)': time_list,
    'Charging Duration': charging_list,
    'Discharging Duration': discharging_list,
    'Idle Duration': idle_list,
    'Change in SoC': change_in_soc_list,
    'Change in SoC kWh': change_in_soc_kwh_list
})

# Replace NaN values with 0 in the 'Change in SoC kWh' column
updated_df['Change in SoC kWh'] = updated_df['Change in SoC kWh'].fillna(0)

# Print the updated dataframe with the desired column order
updated_df = updated_df[['Truck Serial Number', 'SoC', 'SoC in kWh', 'Date', 'Time (24hr)', 'Charging Duration',
                         'Discharging Duration', 'Idle Duration', 'Change in SoC', 'Change in SoC kWh']]




CPU times: user 9min 14s, sys: 2.83 s, total: 9min 17s
Wall time: 9min 22s


In [ ]:
# Replace NaN values with 0
updated_df['Change in SoC'].fillna(0, inplace=True)
updated_df['Change in SoC kWh'].fillna(0, inplace=True)

# Convert negative values to positive values
updated_df['Change in SoC'] = updated_df['Change in SoC'].abs()
updated_df['Change in SoC kWh'] = updated_df['Change in SoC kWh'].abs()

updated_df['Time (24hr)'] = updated_df['Time (24hr)'].dt.time


In [ ]:
null_counts = updated_df.isnull().sum()
print(null_counts)

Truck Serial Number     0
SoC                     0
SoC in kWh              0
Date                    0
Time (24hr)             0
Charging Duration       0
Discharging Duration    0
Idle Duration           0
Change in SoC           0
Change in SoC kWh       0
dtype: int64


In [ ]:
# Display the updated dataframe
updated_df.head(10)

,Truck Serial Number,SoC,SoC in kWh,Date,Time (24hr),Charging Duration,Discharging Duration,Idle Duration,Change in SoC,Change in SoC kWh
0,120660,92.85,167.130,2023-05-01,00:00:00,0.0,0.0,0.0,0.00,0.000
1,120660,92.53,166.554,2023-05-01,00:15:00,0.0,15.0,0.0,0.32,0.576
2,120660,92.31,166.158,2023-05-01,00:30:00,0.0,15.0,0.0,0.22,0.396
3,120660,92.10,165.780,2023-05-01,00:45:00,0.0,15.0,0.0,0.21,0.378
4,120660,91.90,165.420,2023-05-01,01:00:00,0.0,15.0,0.0,0.20,0.360
5,120660,91.68,165.024,2023-05-01,01:15:00,0.0,15.0,0.0,0.22,0.396
6,120660,90.67,163.206,2023-05-01,01:30:00,0.0,15.0,0.0,1.01,1.818
7,120660,89.22,160.596,2023-05-01,01:45:00,0.0,15.0,0.0,1.45,2.610
8,120660,88.10,158.580,2023-05-01,01:55:00,0.0,10.0,0.0,1.12,2.016
9,120660,87.60,157.680,2023-05-01,02:00:00,0.0,5.0,0.0,0.50,0.900


In [ ]:
updated_df.tail(10)

,Truck Serial Number,SoC,SoC in kWh,Date,Time (24hr),Charging Duration,Discharging Duration,Idle Duration,Change in SoC,Change in SoC kWh
1670259,151829,58.38,105.084,2023-05-31,21:48:00,15.0,0.0,0.0,2.13,3.834
1670260,151829,60.51,108.918,2023-05-31,22:03:00,15.0,0.0,0.0,2.13,3.834
1670261,151829,62.65,112.770,2023-05-31,22:18:00,15.0,0.0,0.0,2.14,3.852
1670262,151829,64.83,116.694,2023-05-31,22:33:00,15.0,0.0,0.0,2.18,3.924
1670263,151829,66.98,120.564,2023-05-31,22:48:00,15.0,0.0,0.0,2.15,3.870
1670264,151829,69.11,124.398,2023-05-31,23:03:00,15.0,0.0,0.0,2.13,3.834
1670265,151829,71.25,128.250,2023-05-31,23:18:00,15.0,0.0,0.0,2.14,3.852
1670266,151829,73.45,132.210,2023-05-31,23:33:00,15.0,0.0,0.0,2.20,3.960
1670267,151829,75.65,136.170,2023-05-31,23:48:00,15.0,0.0,0.0,2.20,3.960
1670268,151829,76.57,137.826,2023-05-31,23:55:00,7.0,0.0,0.0,0.92,1.656


In [ ]:
%%time
# Create new columns 'RoC Per Hr' and 'RoCg Per Hr' and initialize them with zeros
updated_df['RoC Per Hr'] = 0
updated_df['RoCg Per Hr'] = 0

# Iterate over the rows of the DataFrame
for index, row in updated_df.iterrows():
    if row['Change in SoC kWh'] != 0:
        if row['Discharging Duration'] > 0:
            updated_df.at[index, 'RoC'] = row['Change in SoC kWh'] / row['Discharging Duration']
            updated_df.at[index, 'RoC Per Hr'] = updated_df.at[index, 'RoC'] * 60  # Convert to kWh per hour
        elif row['Charging Duration'] > 0:
            updated_df.at[index, 'RoCg'] = row['Change in SoC kWh'] / row['Charging Duration']
            updated_df.at[index, 'RoCg Per Hr'] = updated_df.at[index, 'RoCg'] * 60  # Convert to kWh per hour
    elif row['Idle Duration'] > 0:
        updated_df.at[index, 'RoC'] = 0
        updated_df.at[index, 'RoCg'] = 0

# Forward fill the values in 'RoC' and 'RoCg' columns for each unique truck serial number and date
updated_df['RoC'] = updated_df.groupby(['Truck Serial Number', 'Date'])['RoC'].fillna(method='ffill')
updated_df['RoCg'] = updated_df.groupby(['Truck Serial Number', 'Date'])['RoCg'].fillna(method='ffill')

# Replace any NaN values in 'RoC' and 'RoCg' columns with zeros
updated_df['RoC'] = updated_df['RoC'].fillna(0)
updated_df['RoCg'] = updated_df['RoCg'].fillna(0)



CPU times: user 3min 57s, sys: 1.59 s, total: 3min 58s
Wall time: 4min 1s


In [ ]:
updated_df.head(50)

,Truck Serial Number,SoC,SoC in kWh,Date,Time (24hr),Charging Duration,Discharging Duration,Idle Duration,Change in SoC,Change in SoC kWh,RoC Per Hr,RoCg Per Hr,RoC,RoCg
0,120660,92.85,167.130,2023-05-01,00:00:00,0.0,0.0,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000
1,120660,92.53,166.554,2023-05-01,00:15:00,0.0,15.0,0.0,0.32,0.576,2.304000,0.000000,0.038400,0.000000
2,120660,92.31,166.158,2023-05-01,00:30:00,0.0,15.0,0.0,0.22,0.396,1.584000,0.000000,0.026400,0.000000
3,120660,92.10,165.780,2023-05-01,00:45:00,0.0,15.0,0.0,0.21,0.378,1.512000,0.000000,0.025200,0.000000
4,120660,91.90,165.420,2023-05-01,01:00:00,0.0,15.0,0.0,0.20,0.360,1.440000,0.000000,0.024000,0.000000
5,120660,91.68,165.024,2023-05-01,01:15:00,0.0,15.0,0.0,0.22,0.396,1.584000,0.000000,0.026400,0.000000
6,120660,90.67,163.206,2023-05-01,01:30:00,0.0,15.0,0.0,1.01,1.818,7.272000,0.000000,0.121200,0.000000
7,120660,89.22,160.596,2023-05-01,01:45:00,0.0,15.0,0.0,1.45,2.610,10.440000,0.000000,0.174000,0.000000
8,120660,88.10,158.580,2023-05-01,01:55:00,0.0,10.0,0.0,1.12,2.016,12.096000,0.000000,0.201600,0.000000
9,120660,87.60,157.680,2023-05-01,02:00:00,0.0,5.0,0.0,0.50,0.900,10.800000,0.000000,0.180000,0.000000


In [ ]:
updated_df.tail(10)

,Truck Serial Number,SoC,SoC in kWh,Date,Time (24hr),Charging Duration,Discharging Duration,Idle Duration,Change in SoC,Change in SoC kWh,RoC Per Hr,RoCg Per Hr,RoC,RoCg
1670259,151829,58.38,105.084,2023-05-31,21:48:00,15.0,0.0,0.0,2.13,3.834,0.0,15.336000,0.096923,0.255600
1670260,151829,60.51,108.918,2023-05-31,22:03:00,15.0,0.0,0.0,2.13,3.834,0.0,15.336000,0.096923,0.255600
1670261,151829,62.65,112.770,2023-05-31,22:18:00,15.0,0.0,0.0,2.14,3.852,0.0,15.408000,0.096923,0.256800
1670262,151829,64.83,116.694,2023-05-31,22:33:00,15.0,0.0,0.0,2.18,3.924,0.0,15.696000,0.096923,0.261600
1670263,151829,66.98,120.564,2023-05-31,22:48:00,15.0,0.0,0.0,2.15,3.870,0.0,15.480000,0.096923,0.258000
1670264,151829,69.11,124.398,2023-05-31,23:03:00,15.0,0.0,0.0,2.13,3.834,0.0,15.336000,0.096923,0.255600
1670265,151829,71.25,128.250,2023-05-31,23:18:00,15.0,0.0,0.0,2.14,3.852,0.0,15.408000,0.096923,0.256800
1670266,151829,73.45,132.210,2023-05-31,23:33:00,15.0,0.0,0.0,2.20,3.960,0.0,15.840000,0.096923,0.264000
1670267,151829,75.65,136.170,2023-05-31,23:48:00,15.0,0.0,0.0,2.20,3.960,0.0,15.840000,0.096923,0.264000
1670268,151829,76.57,137.826,2023-05-31,23:55:00,7.0,0.0,0.0,0.92,1.656,0.0,14.194286,0.096923,0.236571


In [ ]:
updated_df.shape

(1670269, 14)

In [ ]:
updated_df.columns

Index(['Truck Serial Number', 'SoC', 'SoC in kWh', 'Date', 'Time (24hr)',
       'Charging Duration', 'Discharging Duration', 'Idle Duration',
       'Change in SoC', 'Change in SoC kWh', 'RoC Per Hr', 'RoCg Per Hr',
       'RoC', 'RoCg'],
      dtype='object')

In [ ]:
null_counts = updated_df.isnull().sum()
print(null_counts)

Truck Serial Number     0
SoC                     0
SoC in kWh              0
Date                    0
Time (24hr)             0
Charging Duration       0
Discharging Duration    0
Idle Duration           0
Change in SoC           0
Change in SoC kWh       0
RoC Per Hr              0
RoCg Per Hr             0
RoC                     0
RoCg                    0
dtype: int64


In [ ]:
updated_df.dtypes

Truck Serial Number       int64
SoC                     float64
SoC in kWh              float64
Date                     object
Time (24hr)              object
Charging Duration       float64
Discharging Duration    float64
Idle Duration           float64
Change in SoC           float64
Change in SoC kWh       float64
RoC Per Hr              float64
RoCg Per Hr             float64
RoC                     float64
RoCg                    float64
dtype: object

In [ ]:
# Check memory usage in bytes
memory_usage_bytes = updated_df.memory_usage().sum()

# Convert to kilobytes (KB)
memory_usage_kb = memory_usage_bytes / 1024

# Convert to megabytes (MB)
memory_usage_mb = memory_usage_kb / 1024

print("Memory usage: {:.2f} MB".format(memory_usage_mb))


Memory usage: 178.40 MB


In [ ]:
# Store the modified dataframe in a new CSV file
#updated_df.to_csv('/content/drive/MyDrive/ORANGEEV FILES/May_SOC_Final.csv', index=False)

In [ ]:
###########Creating an Aggregated df for each truck and each date##############

In [ ]:
%%time
##PERFECT#####

# Group the original DataFrame by 'Truck Serial Number' and 'Date' and calculate the sum and mean
grouped_df = updated_df.groupby(['Truck Serial Number', 'Date']).agg({
    'SoC': 'last',
    'SoC in kWh': 'last',
    'Charging Duration': 'sum',
    'Discharging Duration': 'sum',
    'Idle Duration': 'sum',
    'RoC': 'mean',
    'RoCg': 'mean',
    'Time (24hr)': 'last',
    'RoC Per Hr': 'mean',
    'RoCg Per Hr': 'mean'
})

# Reset the index of the grouped DataFrame
grouped_df.reset_index(inplace=True)

# Replace values >= 1434 in 'Charging Duration', 'Discharging Duration', and 'Idle Duration' with 1440
columns_to_replace = ['Charging Duration', 'Discharging Duration', 'Idle Duration']
grouped_df[columns_to_replace] = grouped_df[columns_to_replace].apply(lambda x: np.where(x >= 1434, 1440, x))

# Add a new column named 'Day' that assigns the day of the week based on the 'Date' column
grouped_df['Day'] = pd.to_datetime(grouped_df['Date']).dt.day_name()

# Check the sum of 'Charging Duration', 'Discharging Duration', and 'Idle Duration'
grouped_df['Duration Sum'] = grouped_df[['Charging Duration', 'Discharging Duration', 'Idle Duration']].sum(axis=1)

# Adjust 'Idle Duration' if the sum is not equal to 1440
sum_threshold = 1440
grouped_df['Idle Duration'] = np.where(grouped_df['Duration Sum'] != sum_threshold,
                                      grouped_df['Idle Duration'] + sum_threshold - grouped_df['Duration Sum'],
                                      grouped_df['Idle Duration'])

# Reorder the columns
SoC_Summary = grouped_df[[
    'Truck Serial Number', 'SoC', 'SoC in kWh', 'Date', 'Day', 'Time (24hr)', 'Charging Duration',
    'Discharging Duration', 'Idle Duration', 'RoC', 'RoC Per Hr', 'RoCg', 'RoCg Per Hr'
]].copy()

# Fill any missing values in 'RoC' and 'RoCg' with 0
SoC_Summary['RoC'].fillna(0, inplace=True)
SoC_Summary['RoCg'].fillna(0, inplace=True)

# Replace all infinite values 'inf' with zero
SoC_Summary.replace(float('inf'), 0, inplace=True)

# Check for null values
null_counts = SoC_Summary.isnull().sum()
print(null_counts)


Truck Serial Number     0
SoC                     0
SoC in kWh              0
Date                    0
Day                     0
Time (24hr)             0
Charging Duration       0
Discharging Duration    0
Idle Duration           0
RoC                     0
RoC Per Hr              0
RoCg                    0
RoCg Per Hr             0
dtype: int64
CPU times: user 728 ms, sys: 12 ms, total: 740 ms
Wall time: 758 ms


In [ ]:
SoC_Summary.shape

(21390, 13)

In [ ]:
SoC_Summary.columns

Index(['Truck Serial Number', 'SoC', 'SoC in kWh', 'Date', 'Day',
       'Time (24hr)', 'Charging Duration', 'Discharging Duration',
       'Idle Duration', 'RoC', 'RoC Per Hr', 'RoCg', 'RoCg Per Hr'],
      dtype='object')

In [ ]:
SoC_Summary.head(31)

,Truck Serial Number,SoC,SoC in kWh,Date,Day,Time (24hr),Charging Duration,Discharging Duration,Idle Duration,RoC,RoC Per Hr,RoCg,RoCg Per Hr
0,120660,99.13,178.434,2023-05-01,Monday,23:58:00,636.0,695.0,109.0,0.097382,4.946528,0.068601,2.302931
1,120660,94.16,169.488,2023-05-02,Tuesday,23:50:00,584.0,626.0,230.0,0.087665,4.317507,0.069027,2.221636
2,120660,88.43,159.174,2023-05-03,Wednesday,23:55:00,485.0,805.0,150.0,0.098958,5.381336,0.052791,1.347092
3,120660,96.43,173.574,2023-05-04,Thursday,23:47:00,675.0,564.0,201.0,0.105103,4.845112,0.070664,2.482516
4,120660,87.47,157.446,2023-05-05,Friday,23:42:00,199.0,588.0,653.0,0.093977,4.981797,0.067700,2.136630
5,120660,81.80,147.240,2023-05-06,Saturday,02:17:00,7.0,111.0,1322.0,0.104800,5.448000,0.031929,1.915714
6,120660,91.66,164.988,2023-05-07,Sunday,23:49:00,0.0,165.0,1275.0,0.071486,4.289143,0.000000,0.000000
7,120660,88.57,159.426,2023-05-08,Monday,23:50:00,455.0,470.0,515.0,0.092283,4.616447,0.071044,2.180039
8,120660,93.21,167.778,2023-05-09,Tuesday,23:52:00,531.0,636.0,273.0,0.087638,4.549213,0.048604,1.603201
9,120660,90.24,162.432,2023-05-10,Wednesday,23:58:00,631.0,626.0,183.0,0.102000,5.029561,0.044518,1.838063


In [ ]:
SoC_Summary.tail(31)

,Truck Serial Number,SoC,SoC in kWh,Date,Day,Time (24hr),Charging Duration,Discharging Duration,Idle Duration,RoC,RoC Per Hr,RoCg,RoCg Per Hr
21359,151829,76.31,137.358,2023-05-01,Monday,23:55:00,876.0,0.0,564.0,0.000000,0.000000,0.196506,11.790337
21360,151829,72.37,130.266,2023-05-02,Tuesday,23:55:00,589.0,357.0,494.0,0.115558,6.933450,0.134559,6.630245
21361,151829,52.65,94.770,2023-05-03,Wednesday,23:55:00,527.0,606.0,307.0,0.144789,7.213698,0.159097,6.393584
21362,151829,48.30,86.940,2023-05-04,Thursday,23:55:00,826.0,486.0,128.0,0.134074,6.232589,0.162261,5.967689
21363,151829,62.23,112.014,2023-05-05,Friday,23:55:00,141.0,43.0,1256.0,0.063801,3.828075,0.128322,7.032522
21364,151829,100.00,180.000,2023-05-06,Saturday,23:55:00,354.0,73.0,1013.0,0.033146,1.988762,0.135456,6.882889
21365,151829,55.59,100.062,2023-05-07,Sunday,23:55:00,0.0,482.0,958.0,0.112144,6.728622,0.000000,0.000000
21366,151829,55.59,100.062,2023-05-08,Monday,23:55:00,0.0,0.0,1440.0,0.000000,0.000000,0.000000,0.000000
21367,151829,100.00,180.000,2023-05-09,Tuesday,23:59:00,360.0,8.0,1072.0,0.006398,0.383906,0.177175,10.630500
21368,151829,83.12,149.616,2023-05-10,Wednesday,23:55:00,1.0,250.0,1189.0,0.076907,4.393600,0.003600,0.216000


In [ ]:
unique_serial_numbers = len(SoC_Summary['Truck Serial Number'].unique())
print("Unique number of truck serial numbers:", unique_serial_numbers)


Unique number of truck serial numbers: 810


In [ ]:
# Check memory usage in bytes
memory_usage_bytes = SoC_Summary.memory_usage().sum()

# Convert to kilobytes (KB)
memory_usage_kb = memory_usage_bytes / 1024

# Convert to megabytes (MB)
memory_usage_mb = memory_usage_kb / 1024

print("Memory usage: {:.3f} MB".format(memory_usage_mb))


Memory usage: 2.122 MB


In [ ]:
# Store the modified dataframe in a new CSV file
SoC_Summary.to_csv('/content/drive/MyDrive/ORANGEEV FILES/May_ROC_Final.csv', index=False)

In [ ]:
###CHECK FOR CRITICAL BATTERY INSTANCES
###BROKEN



